In [1]:
import os
import re
import sys
sys.path.append('.')
import cv2
import math
import time
import scipy
import argparse
import matplotlib
import numpy as np
import pylab as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from collections import OrderedDict
from scipy.ndimage.filters import gaussian_filter
from network.rtpose_vgg import get_model
from network.post import decode_pose
from training.datasets.coco_data.preprocessing import (inception_preprocess,
                                              rtpose_preprocess,
                                              ssd_preprocess, vgg_preprocess)
from network import im_transform
from evaluate.coco_eval import get_multiplier, get_outputs, handle_paf_and_heat
from PIL import Image


In [2]:
weight_name = './network/weight/pose_model.pth'
torch.cuda.set_device(0)

In [3]:
weight_name

'./network/weight/pose_model.pth'

In [4]:
#torch.load(weight_name)

In [5]:
model = get_model('vgg19')     
model.load_state_dict(torch.load(weight_name))
model = torch.nn.DataParallel(model).cuda()
model.float()
model.eval()

Bulding VGG19


DataParallel(
  (module): rtpose_model(
    (model0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace)
      (16): Conv2d(256, 256, kerne

In [6]:
import os 
imgs_path='/media/athira/CVlab_IITM/CV lab documents/Works/My works/Openpose/pytorch_Realtime_Multi-Person_Pose_Estimation/demo/new'
img_names=os.listdir(imgs_path)
img_names.sort()
torch.cuda.empty_cache()
img_names

['0001C1T0001F001.jpg',
 '0001C1T0001F002.jpg',
 'COCO_val2014_000000000192.jpg',
 'COCO_val2014_000000000241.jpg']

In [7]:
test_image = os.path.join(imgs_path,img_names[1])
Img = cv2.imread(test_image)
Img.shape

(256, 128, 3)

In [8]:
for im_name in img_names:
    test_image = os.path.join(imgs_path,im_name)
    oriImg = cv2.imread(test_image) # B,G,R order
    shape_dst = np.min(oriImg.shape[0:2])

    # Get results of original image
    multiplier = get_multiplier(oriImg)
    print(im_name, oriImg.shape)

    with torch.no_grad():
        orig_paf, orig_heat = get_outputs(multiplier, oriImg, model,  'rtpose')
        torch.cuda.empty_cache()
          
        # Get results of flipped image
        swapped_img = oriImg[:, ::-1, :]
        flipped_paf, flipped_heat = get_outputs(multiplier, swapped_img,
                                                model, 'rtpose')
        torch.cuda.empty_cache()

        # compute averaged heatmap and paf
        paf, heatmap = handle_paf_and_heat(
            orig_heat, flipped_heat, orig_paf, flipped_paf)
            
    param = {'thre1': 0.1, 'thre2': 0.05, 'thre3': 0.5}
    canvas, to_plot, candidate, subset = decode_pose(
        oriImg, param, heatmap, paf)
    
    
    cv2.imwrite('./demo/new/result_'+  im_name + '.png',to_plot)
    
#     cv2.imwrite("result_%d.jpg" %im_name, image)
#     cv2.imwrite('resultNew.png',to_plot)




0001C1T0001F001.jpg (256, 128, 3)
0001C1T0001F002.jpg (256, 128, 3)
COCO_val2014_000000000192.jpg (480, 640, 3)
COCO_val2014_000000000241.jpg (640, 480, 3)


In [9]:
#torch.cuda.current_device()